In [47]:
import torch
import torch.nn as nn
from  sklearn.datasets import fetch_olivetti_faces 
# 批次加载数据
from torch.utils.data import DataLoader , TensorDataset 
#转化为向量
from torchvision.transforms.v2 import ToTensor 
#拆分函数
from sklearn.model_selection import train_test_split


In [66]:
class TorchNN(nn.Module):
    # 初始化
    def __init__(self):  # self 指代新创建模型对象
        super().__init__()

        self.linear1 = nn.Linear(4096, 512)
        #归一化
        self.bn1 = nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.linear3 = nn.Linear(512, 40)
        #正则话
        self.drop = nn.Dropout(p=0.3)
        self.act = nn.ReLU()

    # forward 前向运算 (nn.Module方法重写)
    def forward(self, input_tensor):
        out = self.linear1(input_tensor)
        out = self.bn1(out)
        out = self.act(out)
        out = self.drop(out)
        out = self.linear2(out)
        out = self.bn2(out)
        out = self.act(out)
        out = self.drop(out)
        final = self.linear3(out) # shape

        return final

In [96]:
#学习率
LR = 1e-1  
# 训练周期
epochs = 30
# 批次大小
BATCH_SIZE = 10

In [97]:
#数据导入，训练集与测试集划分
face =fetch_olivetti_faces(data_home='./face_date',shuffle=True)
train_X,test_X, train_y,test_y=train_test_split(face.data,face.target,test_size=0.2,random_state=42)
# 转换为 PyTorch Tensor 格式
train_X_tensor = torch.tensor(train_X)
train_y_tensor = torch.tensor(train_y)
# 创建 PyTorch Dataset 对象
train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [98]:
  #核心函数
model =TorchNN()
# 损失函数&优化器
loss_fn = nn.CrossEntropyLoss()  # 交叉熵损失函数
# 优化器（模型参数更新）
#optimizer = torch.optim.SGD(model.parameters(), lr=LR)
optimizer = torch.optim.SGD(model.parameters(), lr=LR,momentum=0.9)
#optimizer = torch.optim.SGD(model.parameters(), lr=LR,momentum=0.9, nesterov=True)
#optimizer_adam = torch.optim.Adam(model.parameters(), lr=1e-3)
#optimizer_rmsprop = torch.optim.RMSprop(model.parameters(), lr=1e-3)

In [99]:
for epoch in range(epochs):
    # 提取训练数据
    for data, target in train_dl:
        # 前向运算
        output = model(data)
        # 计算损失
        loss = loss_fn(output, target)
        # 反向传播
        model.zero_grad()  # 所有参数梯度清零
        loss.backward()     # 计算梯度（参数.grad）
        optimizer.step()    # 更新参数
    print(f'Epoch:{epoch} Loss: {loss.item()}')

Epoch:0 Loss: 8.726295471191406
Epoch:1 Loss: 4.149713039398193
Epoch:2 Loss: 1.471557855606079
Epoch:3 Loss: 1.5062861442565918
Epoch:4 Loss: 0.5195940136909485
Epoch:5 Loss: 0.29590046405792236
Epoch:6 Loss: 1.6169488430023193
Epoch:7 Loss: 1.3594567775726318
Epoch:8 Loss: 0.5680615305900574
Epoch:9 Loss: 1.8261350393295288
Epoch:10 Loss: 0.09815323352813721
Epoch:11 Loss: 0.0016433664131909609
Epoch:12 Loss: 1.0548722743988037
Epoch:13 Loss: 0.5640944838523865
Epoch:14 Loss: 0.8294516801834106
Epoch:15 Loss: 0.3266984522342682
Epoch:16 Loss: 0.024898501113057137
Epoch:17 Loss: 0.20863310992717743
Epoch:18 Loss: 0.0035601588897407055
Epoch:19 Loss: 0.7695937752723694
Epoch:20 Loss: 0.7142073512077332
Epoch:21 Loss: 0.5360990762710571
Epoch:22 Loss: 0.012976089492440224
Epoch:23 Loss: 0.9208377599716187
Epoch:24 Loss: 0.9352172017097473
Epoch:25 Loss: 1.1191473007202148
Epoch:26 Loss: 0.37058359384536743
Epoch:27 Loss: 0.3616981506347656
Epoch:28 Loss: 0.9810177683830261
Epoch:29 Loss

In [100]:
#测试
# 转换为 PyTorch Tensor 格式
test_X_tensor = torch.tensor(test_X)
test_tensor = torch.tensor(test_y)
# 创建 PyTorch Dataset 对象
test_dataset = TensorDataset(test_X_tensor, test_tensor)
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

correct = 0
total = 0
with torch.no_grad():  
    for data, target in test_dl:
        output = model(data)
        _, predicted = torch.max(output, 1)  # 返回每行最大值和索引
        total += target.size(0)  # size(0) 等效 shape[0]
        correct += (predicted == target).sum().item()

print(f'Accuracy: {correct/total*100}%')

Accuracy: 76.25%
